In [2]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import random

# 关于 word-embedding，以序列建模为例
# 考虑 source-sentence 和 target-sentence （暂时考虑离散结果）
# 首先构建序列，序列的字符以其在词表中的索引的形式表示出来，首先构建source序列和target序列

# seed = 666
# torch.manual_seed(seed)


# 原序列目标单词的最大原子数（单词表大小）
max_num_src_words = 8
max_num_tgt_words = 8


# 定义特征大小 原API中是512
model_dim = 8


# 定义单词表的最大序列长度
max_src_seq_len = 5
max_tgt_seq_len = 5
max_position_len = 5

batch_size = 2 # 原序列的大小
# src_len = torch.randint(2, 5, (batch_size, ))  # 参数：最小值，最大值，数据类型（数据格式）
# tgt_len = torch.randint(2, 5, (batch_size, ))
src_len = torch.Tensor([2, 4]).to(torch.int32)  # 我们拟造的原序列的长度
tgt_len = torch.Tensor([4, 3]).to(torch.int32)  # 定义的目标序列长度

# print(src_len)  # tensor([2, 4], dtype=torch.int32)
# print(tgt_len)  # tensor([4, 3], dtype=torch.int32)


# 生成原序列 这是以单词索引构成的句子 构建batch
src_seq = torch.cat([torch.unsqueeze(F.pad(torch.randint(1, max_num_src_words, (L,)), (0, max(src_len) - L)), 0) for L in src_len]) 
# [tensor([3, 3, 0, 0, 0]), tensor([2, 2, 7, 3, 0])]  这只是第一步我们要做的，然后就是将两个独立的tensor合并成一个tensor，使用到了cat
# 合并，然后对于每一个Tensor我们在第0维升维  tensor([[3, 5, 0, 0, 0], [3, 6, 4, 1, 0]])


# 生成目标序列 并且按照原序列的形式进行padding 默认值是0
tgt_seq = torch.cat([torch.unsqueeze(F.pad(torch.randint(1, max_num_tgt_words, (L,)), (0, max(src_len) - L)), 0) for L in tgt_len])  
# tensor([[1, 6, 1, 4, 0], [1, 5, 4, 0, 0]])

# print(src_seq)
# print(tgt_seq)


# 构造embedding （source embedding 和target embedding）
# 在这里+1的含义是我们padding了0，所以要预留出0的dim
src_embedding_table = nn.Embedding(max_num_src_words +1, model_dim)
tgt_embedding_table = nn.Embedding(max_num_tgt_words +1, model_dim)

src_embedding = src_embedding_table(src_seq)
tgt_embedding = tgt_embedding_table(tgt_seq)


# 构造position embedding
position_matrix = torch.arange(max_position_len).reshape((-1, 1))  # PE 两大参数
i_matrix = torch.pow(10000, torch.arange(0, model_dim, 2).reshape((1, -1))/model_dim)

pe_embedding_table = torch.zeros(max_position_len, model_dim)
pe_embedding_table[:, ::2] = torch.sin(position_matrix / i_matrix)  # PE公式中偶数列sin的构建
pe_embedding_table[:, 1::2] = torch.cos(position_matrix / i_matrix)  # # PE公式中奇数列cos的构建

# 可以利用nn.enbedding来实现快速构建效果
pe_embedding = nn.Embedding(max_position_len, model_dim)
pe_embedding.weight = nn.Parameter(pe_embedding_table, requires_grad=False)
# print(pe_embedding_table)  # 结果应该是和上面一致的

src_position = torch.cat([torch.unsqueeze(torch.arange(max(src_len)), 0) for _ in src_len]).to(torch.int32)
tgt_position = torch.cat([torch.unsqueeze(torch.arange(max(tgt_len)), 0) for _ in tgt_len]).to(torch.int32)

# print(src_position)
src_pe_embedding = pe_embedding(src_position)
tgt_pe_embedding = pe_embedding(tgt_position)

# print(src_pe_embedding)  # 这结果的size是2*4*8 意味着batch_size是2 sequence_length是4 dim是8
# print(tgt_pe_embedding)

# softmax demo
# 首先随机生成一个正态分布来观察分数
alpha1 = 0.1
alpha2 = 10  # fine-tune 的可能影响
score = torch.randn(5) # 假设是Attention(Q, K, V)中QK点集的结果
prob1 = F.softmax(score*alpha1, -1)
prob2 = F.softmax(score*alpha2, -1)
def soft_max(score):
    return F.softmax(score)
# 查看雅可比矩阵
# jacobian1 = torch.autograd.functional.jacobian(soft_max, score*alpha1)
# jacobian2 = torch.autograd.functional.jacobian(soft_max, score*alpha2)
# print(jacobian1) # alpha=1 的时候梯度比较稳定没有消失
# print(jacobian2)  # 好多地方都要变成了0，梯度消失
# print(prob1)  # tensor([0.1489, 0.1668, 0.0710, 0.4117, 0.2015]) Prob越大代表着两个单词之间的相似性越大
# print(prob2)  # tensor([4.7223e-08, 2.4097e-08, 3.3442e-04, 4.3119e-05, 9.9962e-01]) 

# 构造encoder self-attention mask   mask的shape:[batch_size, max_src_len, max_src_len]，值为1或-inf
# 首先构建有效的编码器
valid_encoder_position = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L), (0, max(src_len)-L)), 0) for L in src_len]), 2)
# print(valid_encoder_position)
# print(valid_encoder_position.shape)  # torch.Size([2, 4, 1]) 这个size的含义是batch_size * padding_sentence_length * padding 


valid_encoder_position_matrix = torch.bmm(valid_encoder_position, valid_encoder_position.transpose(1, 2))  # [2, 4, 1] * [2, 1, 4]  4*4
# print(valid_encoder_position_matrix.shape)
# print(valid_encoder_position_matrix)  # torch.Size([2, 4, 4])


# 创建无效矩阵invalid 然后相减就变成了valid
invalid_encoder_position_matrix = 1- valid_encoder_position_matrix
mask_encoder_self_attention = invalid_encoder_position_matrix.to(torch.bool)
# print(mask_encoder_self_attention.shape)  # torch.Size([2, 4, 4])

# 模拟成绩
score = torch.randn(batch_size, max(src_len), max(src_len))
# print(score.shape)  # torch.Size([2, 4, 4])
masked_score = score.masked_fill(mask_encoder_self_attention, -1e10)
prob = F.softmax(masked_score, -1)

print(src_len)
print(score)
print(masked_score)
print(prob)


tensor([2, 4], dtype=torch.int32)
tensor([[[-0.8338, -0.3374, -1.3284, -1.8859],
         [ 0.6218,  0.1829, -0.4228,  0.1953],
         [ 1.4680, -0.3327, -0.3261, -1.9946],
         [ 0.2265,  1.1493,  0.0440, -1.3728]],

        [[ 0.4587,  0.3253,  0.8320, -0.3004],
         [-1.0352,  0.5890, -0.6019,  1.2396],
         [ 0.6743, -1.9684, -0.2546, -0.7317],
         [-1.4023, -0.9044, -0.5729, -0.0953]]])
tensor([[[-8.3381e-01, -3.3742e-01, -1.0000e+10, -1.0000e+10],
         [ 6.2179e-01,  1.8286e-01, -1.0000e+10, -1.0000e+10],
         [-1.0000e+10, -1.0000e+10, -1.0000e+10, -1.0000e+10],
         [-1.0000e+10, -1.0000e+10, -1.0000e+10, -1.0000e+10]],

        [[ 4.5869e-01,  3.2526e-01,  8.3200e-01, -3.0035e-01],
         [-1.0352e+00,  5.8900e-01, -6.0191e-01,  1.2396e+00],
         [ 6.7434e-01, -1.9684e+00, -2.5457e-01, -7.3167e-01],
         [-1.4023e+00, -9.0441e-01, -5.7288e-01, -9.5280e-02]]])
tensor([[[0.3784, 0.6216, 0.0000, 0.0000],
         [0.6080, 0.3920, 0.0000, 0

In [30]:
#  Decoder 中，传入的Query和 Encoder中传出的 Key Value 加权求和作为一个新的表征进行运算
#  首先Key 和 Query计算出一个score ， 经过softmax函数得到一个weight 然后和 Value加权求和
#  反应的是目标序列和原序列的相关性
#  目标序列的length是不一样的，所以涉及到一个mask将原序列中padding关系是匹配的

#  Step 5 : Intra-attention_mask  Q * K^T  shape:[batch_size, tgt_seq_len, src_seq_len]
#  一些上面的基本参数配置: batch_size = 2
#  src_len = torch.Tensor([2, 4]).to(torch.int32)
#  tgt_len = torch.Tensor([4, 3]).to(torch.int32)
# 
# 

valid_encoder_position = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L), (0, max(src_len)-L)), 0) for L in src_len]), 2)
valid_decoder_position = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L), (0, max(tgt_len)-L)), 0) for L in tgt_len]), 2)
# batch_size == 2 两个序列 1. 有效单词 0. padding 单词
# print(valid_encoder_position)
# print(valid_decoder_position)
# print(valid_decoder_position.shape)  # [2, 4, 1]

valid_cross_position_matrix = torch.bmm(valid_decoder_position, valid_encoder_position.transpose(1, 2))  # shape [2, 4, 4]
# print(valid_cross_position.shape)  # torch.Size([2, 4, 4])

# 无效矩阵，意味着我们不需要来计算相关性
invalid_corss_position_matrix = 1 - valid_cross_position_matrix
mask_cross_position = invalid_corss_position_matrix.to(torch.bool)

# print(mask_cross_position)
# tensor([[[False, False,  True,  True],
#          [False, False,  True,  True],
#          [False, False,  True,  True],
#          [False, False,  True,  True]],

#         [[False, False, False, False],
#          [False, False, False, False],
#          [False, False, False, False],
#          [ True,  True,  True,  True]]])


#  Decoder_self_attention 的 mask 实现
#  掩盖矩阵理论上是一个三角矩阵（下三角）

tri_matrix = [torch.tril(torch.ones((L, L))) for L in tgt_len]  # tgt_len [4, 3]
# print(tri_matrix)  # size [4, 4], [3, 3] 都是下三角矩阵
# 具体的含义是，因为decoder预测是masked的，所以batch中所有元素并不是一次性送到层中，下三角矩阵，1的数量每一次加1，
# 代表每一次迭代预测都多放进去一个词向量

# 构建有效position
# F.pad (左边填充数， 右边填充数， 上边填充数， 下边填充数)  首先是给它扩充两个维度
valid_decoder_tri_matrix = torch.cat([torch.unsqueeze( F.pad(torch.tril \
                                            (torch.ones((L, L))), \
                                            (0, max(tgt_len)-L, 0, max(tgt_len)-L)), 0) \
                                      for L in tgt_len])
# 就是说给右侧和下面两个维度全部pad对齐
# print(valid_decoder_tri_matrix)

# tensor([[[1., 0., 0., 0.],
#          [1., 1., 0., 0.],
#          [1., 1., 1., 0.],
#          [1., 1., 1., 1.]],

#         [[1., 0., 0., 0.],
#          [1., 1., 0., 0.],
#          [1., 1., 1., 0.],
#          [0., 0., 0., 0.]]])

invalid_decoder_tri_matrix = 1 - valid_decoder_tri_matrix
invalid_decoder_tri_matrix = invalid_decoder_tri_matrix.to(torch.bool)
# print(invalid_decoder_tri_matrix)

score = torch.randn(batch_size, max(tgt_len), max(tgt_len))
masked_score = score.masked_fill(invalid_decoder_tri_matrix, -1e9)
prob = F.softmax(masked_score, -1)
print(prob)

# 最后一行都是0.25，因为都被mask掉了，所以他们是0.25平均的

# 构建scaled_self_attention
def scaled_dot_product_attention(Q, K, V, attention_mask):
    score = torch.bmm(Q, K.transpose(-2, -1)) / torch.sqrt(model_dim)
    masked_score = score.masked_fill(attention_mask, -1e9)
    prob = F.softmax(masked_score, -1)
    context = torch.bmm(prob, V)
    return context

# shape of Q, K, V: [batch_size*num_head, sequence_length, model_dim/num_head]
# 但是sequence_length的长度在QKV中可能都不一样，我们需要做归一化

tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.6160, 0.3840, 0.0000, 0.0000],
         [0.1338, 0.6086, 0.2577, 0.0000],
         [0.3920, 0.4120, 0.0941, 0.1018]],

        [[1.0000, 0.0000, 0.0000, 0.0000],
         [0.3951, 0.6049, 0.0000, 0.0000],
         [0.0718, 0.6516, 0.2766, 0.0000],
         [0.2500, 0.2500, 0.2500, 0.2500]]])


## Transformer 源码回顾笔记

- \_\_init\_\_ 函数参数: d_model(model_dim), nhead(num_head), encoder_layer, decoder_layer, feedforward_dim
- forward: src原序列词向量, tgt目标序列词向量, src_mask & tgt_mask, memory_mask(intro_mask)
- multi_head_attention_forward: Q, K, V
    - 计算head维度 head_dim
    - Q, K, V embedding
    - encoder/ decoder/ intro mask